In [4]:
import cv2
import numpy as np
import pandas as pd
import keras
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.models import Input, Model


In [ ]:
metadata = pd.read_csv('metadata.csv')

In [ ]:
X = []
y = []

for image in images:
    img = cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2RGB)
    img = preprocess_input(img.astype(float))
    contour = metadata[metadata.image == image].iloc[0]
    mask = create_mask(img.shape, contour)
    img = cv2.resize(img, (224, 224))
    mask = cv2.resize(mask, (32, 32))
    X.append(img)
    y.append(mask)
    
X = np.array(X)
y = np.array(y)

In [ ]:
X_train = X[:1300]
y_train = y[:1300]
X_val = X[1300:1600]
y_val = y[1300:1600]
X_test = X[1600:]
y_test = y[1600:]

train_seed = 777
val_seed = 222

batch_size = 64

In [ ]:
image_generator = ImageDataGenerator(rotation_range=10,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     horizontal_flip=True, vertical_flip=True, zoom_range=0.25)

mask_generator = ImageDataGenerator(rotation_range=10,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     horizontal_flip=True, vertical_flip=True, zoom_range=0.25)

In [ ]:
def get_mask_generator(y, seed):
    for batch in mask_generator.flow(y, batch_size=batch_size, seed=seed):
        yield batch[:, :, :, 0].reshape((batch.shape[0], batch.shape[1]*batch.shape[2]))

In [ ]:
train_generator = zip(image_generator.flow(X_train, batch_size=batch_size, seed=train_seed), 
                      get_mask_generator(y_train, train_seed))

val_generator = zip(image_generator.flow(X_val, batch_size=batch_size, seed=val_seed), 
                      get_mask_generator(y_val, val_seed))


In [ ]:
model_mobilenet = MobileNet(input_shape=(224,224,3), include_top=False)
model_output = GlobalAveragePooling2D()(model_mobilenet.output)
model_output = Dense(32*32, activation='sigmoid')(model_output)
MODEL = Model(input=model_mobilenet.input, output=model_output)

In [ ]:
MODEL.compile(loss='binary_crossentropy', optimizer=Adam(), 
              metrics=['binary_crossentropy', 'accuracy'])

In [ ]:
MODEL.fit_generator(train_generator, steps_per_epoch=len(X_train) / batch_size,
                    epochs=50, verbose=1, 
                    validation_data=val_generator, validation_steps=len(X_val) / batch_size)